# Dreambooth
Notebook implementation by David Bielejeski
Latest information on: https://github.com/JoePenna/Dreambooth-Stable-Diffusion

## Build Environment

In [ ]:
from IPython.display import clear_output
from IPython.utils import io

print("Installing widgets library...")
with io.capture_output() as captured:
    !pip install ipywidgets
clear_output()

from JupyterNotebookHelpers.installer_progress_bar_widget import InstallerProgressBar

show_detailed_output = False
install_commands = [
    "pip install huggingface_hub",
    "pip install gitpython",
    "pip install -r requirements.txt",
]

progress_bar = InstallerProgressBar()
progress_bar.show(install_commands)

try:
    for i, install_command in enumerate(install_commands):
        if show_detailed_output:
            !{install_command}
        else:
            with io.capture_output() as captured:
                !{install_command}
            print(f"☑ '{install_command.replace('pip install ', '')}' installation complete.")

        # update the progress bar
        progress_bar.increment(i)

    progress_bar.close()
    print("✅ All Dependencies successfully installed.  Proceed to model download.")

except:
    progress_bar.close()
    print("❌ Error installing dependencies")

# Download a 1.* model

In [ ]:
from JupyterNotebookHelpers.download_model import DownloadModel

download_model = DownloadModel()
download_model.show_form()

## Dreambooth Training Environment Setup

In [ ]:
from JupyterNotebookHelpers.setup_training import SetupTraining

setup_training = SetupTraining()

# Pass our selected model to the training form
model_to_train_on = download_model.get_selected_model()

# Show the form.
setup_training.show_form(
    selected_model = model_to_train_on
)

## Training

If training a person or subject, keep an eye on your project's `logs/{folder}/images/train/samples_scaled_gs-00xxxx` generations.

If training a style, keep an eye on your project's `logs/{folder}/images/train/samples_gs-00xxxx` generations.

In [ ]:
from JupyterNotebookHelpers.joe_penna_dreambooth_config import parse_config_file, JoePennaDreamboothConfigSchemaV1
from IPython.display import clear_output
import os
import shutil

clear_output()

# Load the config generated on the previous step
active_config_path = "./joepenna-dreambooth-configs/active-config.json"
config: JoePennaDreamboothConfigSchemaV1 = parse_config_file(active_config_path)

if config is not None:
    # Show the loaded config.
    print("✅ Loaded the config file, proceeding to training")
    print("active-config.json")
    print("------------------")
    print(config.toJSON())

    reg_data_root = f"/workspace/Dreambooth-Stable-Diffusion/regularization_images/{config.dataset}"

    # Cleanup garbage files if they exist
    leftover_training_files = ["./training_images/.ipynb_checkpoints", "./regularization_images/.ipynb_checkpoints"]
    for i, ipynb_checkpoint_folder in enumerate(leftover_training_files):
        if os.path.exists(ipynb_checkpoint_folder):
            shutil.rmtree(ipynb_checkpoint_folder)

    # Train
    !python "main.py" \
        --project_name "{config.project_name}" \
        --training_model "{config.model_filename}" \
        --regularization_images "{reg_data_root}" \
        --training_images "/workspace/Dreambooth-Stable-Diffusion/training_images" \
        --max_training_steps {max_training_steps} \
        --class_word "{config.class_word}" \
        --token "{config.token}" \
        --flip_p {config.flip_p_arg} \
        --learning_rate {config.learning_rate} \
        --save_every_x_steps {config.save_every_x_steps}

## Copy and name the checkpoint file(s)

In [ ]:
from JupyterNotebookHelpers.copy_and_name_checkpoints import CopyAndNameCheckpoints

active_config_path = "./joepenna-dreambooth-configs/active-config.json"

copy_and_name_checkpoints = CopyAndNameCheckpoints()
copy_and_name_checkpoints.execute(
    active_config_path=active_config_path,
    output_folder="./trained_models"
)

# Optional - Upload to google drive
* run the following commands in a new `terminal` in the `Dreambooth-Stable-Diffusion` directory
* `chmod +x ./gdrive`
* `./gdrive about`
* `paste your token here after navigating to the link`
* `./gdrive upload trained_models/{file_name.ckpt}`

# Big Important Note!

The way to use your token is `<token> <class>` ie `joepenna person` and not just `joepenna`